In [1]:
import torch
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression

#### Pytorch AutoGrad Implementation of Multiple Linear Regression with multi output

In [2]:
# 7 input 3 output
X, y  = make_regression(n_samples=500,n_features=7,n_informative=7,n_targets=3,bias=25,noise=20,random_state=13)

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=13)

In [4]:
X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train)
X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test)

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
lr = LinearRegression()
lr.fit(X_train,y_train)
y_predLR = lr.predict(X_test)

In [6]:
r2_score(y_test,y_predLR)

0.9816225295381956

In [7]:
lr.coef_

array([[65.62386914, 50.98342248, 91.8704951 , 29.09756306, 59.38887619,
         4.20446472, 54.8564364 ],
       [ 8.6274954 , 61.89337578, 98.08826114, 14.85498841, 29.02945762,
        31.42434961, 58.00218984],
       [86.47409391, 52.53801642, 11.1883771 , 39.63969653, 19.08273696,
        94.83906132, 49.41689617]])

In [8]:
lr.intercept_

array([25.1496413 , 25.50028908, 24.6615854 ])

In [66]:
class AutoGDClass:
    def __init__(self, epochs, lr):
        self.epochs = epochs
        self.lr = lr
        self.intercept = None
        self.coeff = None
    
    def forward(self,X):
        return (X @ self.coeff.T) + self.intercept
    
    def loss(self,y, y_hat):
        return torch.mean((y - y_hat) ** 2)
    
    def fit(self, X_train, y_train):
        features = X_train.shape[1]
        target = y_train.shape[1]
        self.coeff = torch.randn((target,features),dtype=float,requires_grad=True)
        self.intercept = torch.randn((1,target),dtype=float,requires_grad=True)
        for i in range(self.epochs):
            y_hat = self.forward(X_train)
            loss = self.loss(y_train,y_hat)
            loss.backward()
            grad_coeff = self.coeff.grad # derivative of loss func wrt coeff
            grad_intercept = self.intercept.grad # derivatitve of loss func wrt intercept
            with torch.no_grad():
                # Note- self.coeff = self.coeff - self.lr * grad_coeff wont work
                # Bcs it creates a new obj and backprop is gone

                self.coeff -= self.lr * grad_coeff
                self.intercept -= self.lr * grad_intercept
            self.coeff.grad.zero_()
            self.intercept.grad.zero_()
                
    
    def predict(self,X_test):
        y_pred = self.forward(X_test)
        return y_pred

In [67]:
mygd = AutoGDClass(100,0.1)
mygd.fit(X_train, y_train)

In [68]:
y_predGD = mygd.predict(X_test)
r2_score(y_test,y_predGD.detach().numpy())

0.9816655841796346

In [69]:
mygd.coeff

tensor([[65.6614, 50.9275, 91.7333, 29.0377, 59.1711,  4.3611, 54.7110],
        [ 8.7005, 61.8045, 97.9581, 14.7955, 28.8873, 31.4903, 57.8922],
        [86.3651, 52.4501, 11.2902, 39.5310, 19.2164, 94.6375, 49.2910]],
       dtype=torch.float64, requires_grad=True)

In [70]:
mygd.intercept

tensor([[25.0120, 25.4000, 24.7050]], dtype=torch.float64, requires_grad=True)

We can see that the metrics match